In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import balanced_accuracy_score
import os

In [ ]:
# Load knee angle data
results_path_knee_angle = "C:/Users/kubil/Documents/STUDIUM/Master/4_Masterarbeit_Code/IOE-Algo-compare/imt-main/examples/results"
ring_angles_kl = np.load(os.path.join(results_path_knee_angle, "Angles_IMU_RING_Knee_Left.npy"), allow_pickle=True)
ring_angles_kr = np.load(os.path.join(results_path_knee_angle, "Angles_IMU_RING_Knee_Right.npy"), allow_pickle=True)

# Load labels
results_path_label = "C:/Users/kubil/Documents/STUDIUM/Master/4_Masterarbeit_Code/IOE-Algo-compare/Klassifikation/results"
labels_extendedLeg_100 = np.load(os.path.join(results_path_label, "labels_extendedLeg_RING_100.npy"), allow_pickle=True)
labels_stiffMovement_100 = np.load(os.path.join(results_path_label, "labels_stiffMovement_RING_100.npy"), allow_pickle=True)

# Choose which label to use
y_stiffMovement = [entry['behavior_label'] for entry in labels_stiffMovement_100]
# y_extendedLeg = [entry['behavior_label_5s'] for entry in labels_extendedLeg_100]

In [ ]:
X = []
y = []
groups = []

for i in range(52):
    left = ring_angles_kl[i]
    right = ring_angles_kr[i]
    label = y_stiffMovement[i]
    #label = y_extendedLeg[i]

    if len(left) == len(right) == len(label):
        X.append(np.column_stack([left, right]))  # shape (n_i, 2)
        y.append(label)                           # shape (n_i,)
        group_id = i // 2                         # every 2 datasets form one group
        groups.extend([group_id] * len(label))
    else:
        print(f"Skipping dataset {i} due to length mismatch.")

# Final stacked arrays
X = np.vstack(X)            # shape (total_samples, 2)
y = np.concatenate(y)       # shape (total_samples,)
groups = np.array(groups)   # shape (total_samples,)

# Check consistency
print(f"X shape: {X.shape}, y shape: {y.shape}, groups shape: {groups.shape}")

# Initialize classifier
clf = LogisticRegression(
    class_weight='balanced',
    random_state=1,
    max_iter=1000
)

# StratifiedGroupKFold CV
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=1)

# Cross-validation loop
scores = []
for train_idx, test_idx in cv.split(X, y, groups):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    score = balanced_accuracy_score(y_test, y_pred)
    scores.append(score)

# Output results
print("Balanced Accuracy Scores:", scores)
print("Mean Balanced Accuracy:", np.mean(scores))
